# 自製智能中文選字系統  (2)

In [ ]:
import sys
sys.version

'3.7.10 (default, Feb 20 2021, 21:17:23) \n[GCC 7.5.0]'

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


確認版本為 python3

## 資料前處理

In [2]:
import re

In [3]:
def prepocess_line(line):
    chinese_pattern = r'[\u4E00-\u9FFF]+'
    segments = re.findall(chinese_pattern, line, flags=re.UNICODE)
    return segments

In [4]:
segments = []
with open('/content/drive/MyDrive/NLP/wiki_zh_small.txt') as fr:
    for line in fr.readlines():
        segments += prepocess_line(line)

## 斷詞

In [ ]:
!pip install jieba

In [5]:
import jieba

In [6]:
list(jieba.cut_for_search(segments[6001]))

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.044 seconds.
Prefix dict has been built successfully.


['所以', '僅用', '於', '還原', '一些', '貴重', '的', '化合', '化合物']

In [150]:
cut_segments = []
for seg in segments:
    # 使用結巴斷詞的 cut_for_search
    cut_segments+=list((jieba.cut_for_search(seg)))

## 使用斷詞的結果來作Ngram

In [151]:
from collections import Counter

class Counters:
    def __init__(self, n):
        self.n = n
        self.counters = [Counter() for _ in range(n + 1)]

    def fit(self, segments):
        for i in range(1, 1 + self.n):
            for segment in segments:
                self.counters[i] += Counter(self._skip(segment, i))

        base_count = sum(dict(self.counters[1]).values())
        self.counters[0] = Counter({'': base_count})

    def __getitem__(self, k):
        return self.counters[k]

    def _skip(self, segment, n):
        assert n > 0
        if len(segment) < n:
            return []
        shift = n - 1
        for i in range(len(segment) - shift):
            yield segment[i:i+shift+1]

In [152]:
counters = Counters(n=5)
counters.fit(cut_segments)

In [153]:
class Ngram:
    def __init__(self, n, counters):
        assert n <= counters.n
        self.n = n
        self.major_counter = counters[n]
        self.minor_counter = counters[n-1]

    def predict_proba(self, prefix='', top_k=5):
        assert len(prefix) >= self.n - 1

        reference = prefix[-(self.n - 1):] if self.n > 1 else ''
        count_referecne = self.minor_counter[reference]
        probs = []
        for key, count in dict(self.major_counter).items():
            if key.startswith(reference):
                prob = count / count_referecne
                probs.append((prob, key[-1]))
        sorted_probs = sorted(probs, reverse=True)
        return sorted_probs[:top_k] if top_k > 0 else sorted_probs

    def get_proba_dict(self, prefix=''):
        return {word: prob for prob, word in self.predict_proba(prefix, top_k=-1)}

In [154]:
ngrams = [Ngram(i, counters) for i in range(1, 6)]

## 使用Smoothing of Language Models來建立第二版選字系統

In [155]:
class ChineseWordRecommenderV2:
    def __init__(self, ngrams):
        self.ngrams = ngrams
    
    def predict_proba(self, prefix='', top_k=5, how='backoff'):
        proba_dicts = [ngram.get_proba_dict(prefix) for ngram in ngrams[:len(prefix)+1]]

        # 在此你可以選擇兩種 Smoothing of Language Models 的方法：
        # Back-off Smoothing 或 Interpolation Smoothing
        # 如果你選擇 Interpolation Smoothing，你可以運用已經準備好的輔助函式
        # _get_interpolation_proba 來達到此目的
        probas = []
        if how == 'interpolation':
            interpolation_lambda = 0.99
            words = proba_dicts[0].keys() #取出所有字元可能(unigram)
            for word in words: 
                proba = self._get_interpolation_proba(word, proba_dicts, interpolation_lambda)
                probas.append((proba, word))

            sorted_probas = sorted(probas, reverse=True)
            return sorted_probas[:top_k] if top_k > 0 else sorted_probas
        if how == 'backoff':
            alpha = 0.4
            words = proba_dicts[0].keys()
            for word in words:
                proba = self._get_backoff_proba(word, proba_dicts, alpha)
                probas.append((proba, word))
                
            sorted_probas = sorted(probas, reverse=True)
            return sorted_probas[:top_k] if top_k > 0 else sorted_probas
    
    def _get_backoff_proba(self, word, proba_dicts, alpha, idx=None):
        if idx is None:
            idx = len(proba_dicts) -1


        if word in proba_dicts[idx]:
            return proba_dicts[idx].get(word)

        if word not in proba_dicts[idx]:
            return alpha * self._get_backoff_proba(word, proba_dicts, alpha, idx=idx-1)
    
    def _get_interpolation_proba(self, word, proba_dicts, interp_lambda, idx=None):
        if idx is None:
            idx = len(proba_dicts) - 1
        if idx == 0:
            return proba_dicts[idx].get(word, 0.)
        return interp_lambda * proba_dicts[idx].get(word, 0.) + \
               (1 - interp_lambda) * self._get_interpolation_proba(word, proba_dicts, interp_lambda, idx=idx-1)

In [156]:
model = ChineseWordRecommenderV2(ngrams)

In [157]:
probs = model.predict_proba('法', top_k=10, how='interpolation')
probs

[(0.051904146168121834, '國'),
 (0.04388616116650217, '律'),
 (0.029848241612163377, '院'),
 (0.020222220928179165, '語'),
 (0.011414997351123958, '蘭'),
 (0.010549762034510142, '系'),
 (0.00792383560501087, '定'),
 (0.007908712635409567, '制'),
 (0.007072068387587443, '學'),
 (0.007060356996987107, '西')]

In [158]:
probs = model.predict_proba('法', top_k=10, how='backoff')
probs

[(0.05230496453900709, '國'),
 (0.044326241134751775, '律'),
 (0.030141843971631204, '院'),
 (0.020390070921985817, '語'),
 (0.01351799989816182, '的'),
 (0.01152482269503546, '蘭'),
 (0.010638297872340425, '系'),
 (0.007978723404255319, '定'),
 (0.007978723404255319, '制'),
 (0.0070921985815602835, '西')]

## Demo

In [159]:
import ipywidgets as widgets

text = widgets.Textarea()
label = widgets.Label()
display(label, text)

def func(change):
    probs = model.predict_proba(change.new, top_k=10)
    label.value = ' ' + '\t'.join([word for prob, word in probs])

text.observe(func, names='value')

Label(value='')

Textarea(value='')